In [3]:
# Import libraries
import gc
import math
import pandas as pd
import numpy as np
import gensim.models as gsm
import multiprocessing as mp
import datetime as dt

from scipy.spatial.distance import cosine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import log_loss

In [4]:
gc.collect()

7

In [3]:
# Import data
train_lookup_df = pd.read_pickle('./pickles.gi/train_lookup_df.pkl')
questions_df = pd.read_pickle('./pickles.gi/questions_with_tokens_df.pkl')

In [4]:
# Get initial number of rows for processing validation
num_rows_questions_df_1 = questions_df.shape[0]

In [5]:
# Get sentences for vectorization
questions_df['preproc_question'] = [' '.join(token_list) for token_list in questions_df.token_list]

In [6]:
# Get a term vector for each question
vectorizer = TfidfVectorizer()
tfidf_vector = vectorizer.fit_transform(questions_df.preproc_question)
questions_df['tfidf_vector'] = list(tfidf_vector)

In [17]:
# Reshape dataframe from stacked to wide format (pivot on pair ID)
questions_df = questions_df.merge(questions_df, on=['test', 'pid'], how='inner', suffixes=('_1', '_2'))
questions_df = questions_df[(questions_df['qid_1'] != questions_df['qid_2']) & (questions_df['qid_1'] < questions_df['qid_2'])]
questions_df.reset_index(drop=True, inplace=True)
questions_df.sort_values(by=['test', 'pid'], inplace=True)

In [18]:
# Validate that the number of rows * 2 in current DF is equal to imported DF
num_rows_questions_df_2 = questions_df.shape[0]
assert num_rows_questions_df_1 == num_rows_questions_df_2 * 2, "Number of rows in questions_df is not consistent with original import"

In [25]:
questions_df['tfidf_cos_sim'] = questions_df.apply(lambda x: (1 - cosine(x['tfidf_vector_1'].toarray(), x['tfidf_vector_2'].toarray())), axis=1)
questions_df.to_pickle('./pickles.gi/questions_df_tfidf_cosine_similarity')

C:\Users\camro\Anaconda3\lib\site-packages\scipy\spatial\distance.py:326: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [26]:
questions_df.head()

,test,pid,qid_1,question_1,token_list_1,preproc_question_1,tfidf_vector_1,qid_2,question_2,token_list_2,preproc_question_2,tfidf_vector_2,tfidf_cos_sim
0,0,0,1,What is the step by step guide to invest in sh...,"[step, step, guide, invest, share, market, india]",step step guide invest share market india,"(0, 94663)\t0.686112034821\n (0, 45076)\t0....",2,What is the step by step guide to invest in sh...,"[step, step, guide, invest, share, market]",step step guide invest share market,"(0, 94663)\t0.7005065502\n (0, 45076)\t0.41...",0.979451
1,0,1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...,"[story, kohinoor, koh, noor, diamond]",story kohinoor koh noor diamond,"(0, 94981)\t0.28471980451\n (0, 57115)\t0.5...",4,What would happen if the Indian government sto...,"[would, happen, indian, government, stole, koh...",would happen indian government stole kohinoor ...,"(0, 57115)\t0.415778246801\n (0, 57112)\t0....",0.761120
2,0,2,5,How can I increase the speed of my internet co...,"[increase, speed, internet, connection, using,...",increase speed internet connection using vpn,"(0, 50805)\t0.364988290024\n (0, 93488)\t0....",6,How can Internet speed be increased by hacking...,"[internet, speed, increased, hacking, dns]",internet speed increased hacking dns,"(0, 93488)\t0.365133417374\n (0, 52186)\t0....",0.274354
3,0,3,7,Why am I mentally very lonely? How can I solve...,"[mentally, lonely, solve]",mentally lonely solve,"(0, 64269)\t0.617633676496\n (0, 60405)\t0....",8,Find the remainder when [math]23^{24}[/math] i...,"[find, remainder, math, 23, 24, math, divided,...",find remainder math 23 24 math divided 24 23,"(0, 39537)\t0.172910152077\n (0, 84134)\t0....",0.000000
4,0,4,9,"Which one dissolve in water quikly sugar, salt...","[one, dissolve, water, quikly, sugar, salt, me...",one dissolve water quikly sugar salt methane c...,"(0, 72242)\t0.154796417483\n (0, 32331)\t0....",10,Which fish would survive in salt water?,"[fish, would, survive, salt, water]",fish would survive salt water,"(0, 109184)\t0.283396040472\n (0, 107410)\t...",0.230124


In [2]:
# IMPORT CONTINUED WORK
questions_df = pd.read_pickle('./pickles.gi/questions_df_tfidf_cosine_similarity')